In [ ]:
#인공 신경망 (Artificial Neural Networks, ANNs)
#가장 기본적인 형태의 신경망, 다층 퍼셉트론(Multilayer Perceptron, MLP)이라고도 불림. 여러 개의 은닉층과 노드로 구성, 각 노드는 활성화 함수(activation function)를 사용합니다.
#적용 분야: 일반적인 분류 및 회귀 문제

# 따라서, NLP 다중 퍼셉트론 모델로, 이진 분류 구축

In [ ]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score
from keras.models import Sequential
from keras.layers import Dense
from keras.utils import to_categorical
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [57]:
data = pd.read_csv('/content/drive/MyDrive/항공사/train (전처리 완료).csv', encoding = 'cp949')

<ipython-input-57-d3a145148dd5>:1: DtypeWarning: Columns (4,5) have mixed types. Specify dtype option on import or set low_memory=False.
  data = pd.read_csv('/content/drive/MyDrive/항공사/train (전처리 완료).csv', encoding = 'cp949')


In [58]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 854947 entries, 0 to 854946
Data columns (total 17 columns):
 #   Column      Non-Null Count   Dtype  
---  ------      --------------   -----  
 0   Unnamed: 0  854947 non-null  float64
 1   샘플고유ID      854947 non-null  object 
 2   월           854947 non-null  int64  
 3   일           854947 non-null  int64  
 4   출발예측시간      854947 non-null  object 
 5   도착예측시간      854947 non-null  object 
 6   결항여부        854947 non-null  int64  
 7   경유여부        854947 non-null  int64  
 8   출발공항고유코드    854947 non-null  object 
 9   출발공항고유ID    854947 non-null  int64  
 10  출발공항_주      854947 non-null  object 
 11  도착공항고유코드    854947 non-null  object 
 12  도착공항공유ID    854947 non-null  int64  
 13  도착공항_주      854947 non-null  object 
 14  이동거리        854947 non-null  int64  
 15  항공사         854947 non-null  object 
 16  항공편지연여부     218250 non-null  object 
dtypes: float64(1), int64(7), object(9)
memory usage: 110.9+ MB


In [59]:
data = data.drop(['Unnamed: 0'], axis = 1)

In [60]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 854947 entries, 0 to 854946
Data columns (total 16 columns):
 #   Column    Non-Null Count   Dtype 
---  ------    --------------   ----- 
 0   샘플고유ID    854947 non-null  object
 1   월         854947 non-null  int64 
 2   일         854947 non-null  int64 
 3   출발예측시간    854947 non-null  object
 4   도착예측시간    854947 non-null  object
 5   결항여부      854947 non-null  int64 
 6   경유여부      854947 non-null  int64 
 7   출발공항고유코드  854947 non-null  object
 8   출발공항고유ID  854947 non-null  int64 
 9   출발공항_주    854947 non-null  object
 10  도착공항고유코드  854947 non-null  object
 11  도착공항공유ID  854947 non-null  int64 
 12  도착공항_주    854947 non-null  object
 13  이동거리      854947 non-null  int64 
 14  항공사       854947 non-null  object
 15  항공편지연여부   218250 non-null  object
dtypes: int64(7), object(9)
memory usage: 104.4+ MB


In [61]:
data.head(5)

,샘플고유ID,월,일,출발예측시간,도착예측시간,결항여부,경유여부,출발공항고유코드,출발공항고유ID,출발공항_주,도착공항고유코드,도착공항공유ID,도착공항_주,이동거리,항공사,항공편지연여부
0,TRAIN_000001,8,15,300.0,624.0,0,0,ORD,13930,Illinois,SLC,14869,Utah,1250,SkyWest Airlines Inc.,NaN
1,TRAIN_000002,9,6,610.0,1085.0,0,0,CLT,11057,North Carolina,LGA,12953,New York,544,American Airlines Inc.,NaN
2,TRAIN_000003,7,10,345.0,1055.0,0,0,LAX,12892,California,EWR,11618,New Jersey,2454,United Air Lines Inc.,NaN
3,TRAIN_000004,1,11,340.0,619.0,0,0,SFO,14771,California,ACV,10157,California,250,SkyWest Airlines Inc.,NaN
4,TRAIN_000006,1,20,662.0,1143.0,0,0,EWR,11618,New Jersey,BOS,10721,Massachusetts,200,United Air Lines Inc.,Not_Delayed


In [62]:
# '출발예측시간' 열에서 'TRAIN_'을 포함하는 행을 삭제
data = data[~data['출발예측시간'].str.contains('TRAIN_', na=False)]
data = data[~data['도착예측시간'].str.contains('TRAIN_', na=False)]
# 삭제 후 데이터프레임 확인
print(data)

              샘플고유ID  월   일      출발예측시간  도착예측시간  결항여부  경유여부 출발공항고유코드  \
0       TRAIN_000001  8  15       300.0   624.0     0     0      ORD   
1       TRAIN_000002  9   6       610.0  1085.0     0     0      CLT   
2       TRAIN_000003  7  10       345.0  1055.0     0     0      LAX   
3       TRAIN_000004  1  11       340.0   619.0     0     0      SFO   
4       TRAIN_000006  1  20       662.0  1143.0     0     0      EWR   
...              ... ..  ..         ...     ...   ...   ...      ...   
854942  TRAIN_999994  6  26  658.056497  1080.0     0     0      DAL   
854943  TRAIN_999995  9  18       376.0   763.0     0     0      ORD   
854944  TRAIN_999996  5  30       360.0   628.0     0     0      FAR   
854945  TRAIN_999997  6  28       320.0   820.0     0     0      OAK   
854946  TRAIN_999998  9  27       613.0  1104.0     0     0      BNA   

        출발공항고유ID          출발공항_주 도착공항고유코드  도착공항공유ID         도착공항_주  이동거리  \
0          13930        Illinois      SLC     14869        

In [63]:
data[data['출발예측시간']=='TRAIN_177269'] # 삭제 확인

,샘플고유ID,월,일,출발예측시간,도착예측시간,결항여부,경유여부,출발공항고유코드,출발공항고유ID,출발공항_주,도착공항고유코드,도착공항공유ID,도착공항_주,이동거리,항공사,항공편지연여부


In [64]:
# 항공편지연여부 열에서 NaN 값을 포함하는 행을 모두 삭제
data.dropna(subset=['항공편지연여부'], inplace=True)

print(data)

              샘플고유ID   월   일      출발예측시간  도착예측시간  결항여부  경유여부 출발공항고유코드  \
4       TRAIN_000006   1  20       662.0  1143.0     0     0      EWR   
6       TRAIN_000010   8  13       650.0  1124.0     0     0      DCA   
8       TRAIN_000012   1  12       375.0   705.0     0     0      CLE   
9       TRAIN_000013   9  19       255.0   426.0     0     0      MAF   
13      TRAIN_000019   7  14       707.0  1305.0     0     0      BDL   
...              ...  ..  ..         ...     ...   ...   ...      ...   
854912  TRAIN_999955   4  12       510.0   836.0     0     0      SEA   
854917  TRAIN_999962  10  11  727.875676  1203.0     0     0      SAT   
854921  TRAIN_999969  10  10       380.0   656.0     0     0      MFE   
854935  TRAIN_999985   8   8       714.0  1239.0     0     0      RDU   
854941  TRAIN_999992  12  30       507.0  1027.0     0     0      JAC   

        출발공항고유ID          출발공항_주 도착공항고유코드  도착공항공유ID         도착공항_주  이동거리  \
4          11618      New Jersey      BOS     1

In [65]:
# 'Not_delayed'와 'delayed' 이외의 값은 NaN으로 대체하고, NaN 값을 0으로 대체하며, 'Not_delayed'를 0으로, 'delayed'를 1로 매핑하여 열 값 변경
data['항공편지연여부'] = data['항공편지연여부'].map({'Not_delayed': 0, 'Delayed': 1})
data['항공편지연여부'] = data['항공편지연여부'].fillna(0)

print(data)

              샘플고유ID   월   일      출발예측시간  도착예측시간  결항여부  경유여부 출발공항고유코드  \
4       TRAIN_000006   1  20       662.0  1143.0     0     0      EWR   
6       TRAIN_000010   8  13       650.0  1124.0     0     0      DCA   
8       TRAIN_000012   1  12       375.0   705.0     0     0      CLE   
9       TRAIN_000013   9  19       255.0   426.0     0     0      MAF   
13      TRAIN_000019   7  14       707.0  1305.0     0     0      BDL   
...              ...  ..  ..         ...     ...   ...   ...      ...   
854912  TRAIN_999955   4  12       510.0   836.0     0     0      SEA   
854917  TRAIN_999962  10  11  727.875676  1203.0     0     0      SAT   
854921  TRAIN_999969  10  10       380.0   656.0     0     0      MFE   
854935  TRAIN_999985   8   8       714.0  1239.0     0     0      RDU   
854941  TRAIN_999992  12  30       507.0  1027.0     0     0      JAC   

        출발공항고유ID          출발공항_주 도착공항고유코드  도착공항공유ID         도착공항_주  이동거리  \
4          11618      New Jersey      BOS     1

In [66]:
data.항공편지연여부.unique()

array([0., 1.])

In [67]:
data.info()

<class 'pandas.core.frame.DataFrame'>
Index: 218246 entries, 4 to 854941
Data columns (total 16 columns):
 #   Column    Non-Null Count   Dtype  
---  ------    --------------   -----  
 0   샘플고유ID    218246 non-null  object 
 1   월         218246 non-null  int64  
 2   일         218246 non-null  int64  
 3   출발예측시간    218246 non-null  object 
 4   도착예측시간    218246 non-null  object 
 5   결항여부      218246 non-null  int64  
 6   경유여부      218246 non-null  int64  
 7   출발공항고유코드  218246 non-null  object 
 8   출발공항고유ID  218246 non-null  int64  
 9   출발공항_주    218246 non-null  object 
 10  도착공항고유코드  218246 non-null  object 
 11  도착공항공유ID  218246 non-null  int64  
 12  도착공항_주    218246 non-null  object 
 13  이동거리      218246 non-null  int64  
 14  항공사       218246 non-null  object 
 15  항공편지연여부   218246 non-null  float64
dtypes: float64(1), int64(7), object(8)
memory usage: 28.3+ MB


In [68]:
# '출발예측시간'과 '도착예측시간' 열을 float 형태로 변환
data['출발예측시간'] = data['출발예측시간'].astype(float)
data['도착예측시간'] = data['도착예측시간'].astype(float)

print(data)

              샘플고유ID   월   일      출발예측시간  도착예측시간  결항여부  경유여부 출발공항고유코드  \
4       TRAIN_000006   1  20  662.000000  1143.0     0     0      EWR   
6       TRAIN_000010   8  13  650.000000  1124.0     0     0      DCA   
8       TRAIN_000012   1  12  375.000000   705.0     0     0      CLE   
9       TRAIN_000013   9  19  255.000000   426.0     0     0      MAF   
13      TRAIN_000019   7  14  707.000000  1305.0     0     0      BDL   
...              ...  ..  ..         ...     ...   ...   ...      ...   
854912  TRAIN_999955   4  12  510.000000   836.0     0     0      SEA   
854917  TRAIN_999962  10  11  727.875676  1203.0     0     0      SAT   
854921  TRAIN_999969  10  10  380.000000   656.0     0     0      MFE   
854935  TRAIN_999985   8   8  714.000000  1239.0     0     0      RDU   
854941  TRAIN_999992  12  30  507.000000  1027.0     0     0      JAC   

        출발공항고유ID          출발공항_주 도착공항고유코드  도착공항공유ID         도착공항_주  이동거리  \
4          11618      New Jersey      BOS     1

In [69]:
data.info()

<class 'pandas.core.frame.DataFrame'>
Index: 218246 entries, 4 to 854941
Data columns (total 16 columns):
 #   Column    Non-Null Count   Dtype  
---  ------    --------------   -----  
 0   샘플고유ID    218246 non-null  object 
 1   월         218246 non-null  int64  
 2   일         218246 non-null  int64  
 3   출발예측시간    218246 non-null  float64
 4   도착예측시간    218246 non-null  float64
 5   결항여부      218246 non-null  int64  
 6   경유여부      218246 non-null  int64  
 7   출발공항고유코드  218246 non-null  object 
 8   출발공항고유ID  218246 non-null  int64  
 9   출발공항_주    218246 non-null  object 
 10  도착공항고유코드  218246 non-null  object 
 11  도착공항공유ID  218246 non-null  int64  
 12  도착공항_주    218246 non-null  object 
 13  이동거리      218246 non-null  int64  
 14  항공사       218246 non-null  object 
 15  항공편지연여부   218246 non-null  float64
dtypes: float64(3), int64(7), object(6)
memory usage: 28.3+ MB


In [70]:
data.head(3)

,샘플고유ID,월,일,출발예측시간,도착예측시간,결항여부,경유여부,출발공항고유코드,출발공항고유ID,출발공항_주,도착공항고유코드,도착공항공유ID,도착공항_주,이동거리,항공사,항공편지연여부
4,TRAIN_000006,1,20,662.0,1143.0,0,0,EWR,11618,New Jersey,BOS,10721,Massachusetts,200,United Air Lines Inc.,0.0
6,TRAIN_000010,8,13,650.0,1124.0,0,0,DCA,11278,Virginia,PIT,14122,Pennsylvania,204,Republic Airlines,1.0
8,TRAIN_000012,1,12,375.0,705.0,0,0,CLE,11042,Ohio,DEN,11292,Colorado,1201,Southwest Airlines Co.,0.0


In [71]:
data = data.drop(columns=['샘플고유ID'])

In [72]:
from imblearn.over_sampling import SMOTE

In [73]:
data.info()

<class 'pandas.core.frame.DataFrame'>
Index: 218246 entries, 4 to 854941
Data columns (total 15 columns):
 #   Column    Non-Null Count   Dtype  
---  ------    --------------   -----  
 0   월         218246 non-null  int64  
 1   일         218246 non-null  int64  
 2   출발예측시간    218246 non-null  float64
 3   도착예측시간    218246 non-null  float64
 4   결항여부      218246 non-null  int64  
 5   경유여부      218246 non-null  int64  
 6   출발공항고유코드  218246 non-null  object 
 7   출발공항고유ID  218246 non-null  int64  
 8   출발공항_주    218246 non-null  object 
 9   도착공항고유코드  218246 non-null  object 
 10  도착공항공유ID  218246 non-null  int64  
 11  도착공항_주    218246 non-null  object 
 12  이동거리      218246 non-null  int64  
 13  항공사       218246 non-null  object 
 14  항공편지연여부   218246 non-null  float64
dtypes: float64(3), int64(7), object(5)
memory usage: 26.6+ MB


In [81]:
from sklearn.preprocessing import LabelEncoder

# LabelEncoder 객체 생성
label_encoders = {}
columns_to_encode = ['출발공항고유코드', '출발공항_주', '도착공항고유코드', '도착공항_주', '항공사']

# 각 열에 대해 라벨 인코딩 적용
for column in columns_to_encode:
    le = LabelEncoder()
    data[column] = le.fit_transform(data[column])
    label_encoders[column] = le

print(data)

         월   일      출발예측시간  도착예측시간  결항여부  경유여부  출발공항고유코드  출발공항고유ID  출발공항_주  \
4        1  20  662.000000  1143.0     0     0       119     11618      28   
6        8  13  650.000000  1124.0     0     0        93     11278      47   
8        1  12  375.000000   705.0     0     0        72     11042      33   
9        9  19  255.000000   426.0     0     0       214     13158      42   
13       7  14  707.000000  1305.0     0     0        30     10529       6   
...     ..  ..         ...     ...   ...   ...       ...       ...     ...   
854912   4  12  510.000000   836.0     0     0       319     14747      48   
854917  10  11  727.875676  1203.0     0     0       309     14683      42   
854921  10  10  380.000000   656.0     0     0       222     13256      42   
854935   8   8  714.000000  1239.0     0     0       295     14492      31   
854941  12  30  507.000000  1027.0     0     0       180     12441      51   

        도착공항고유코드  도착공항공유ID  도착공항_주  이동거리  항공사  항공편지연여부  
4     

In [82]:
X = data.iloc[:,:-1]
y = data.iloc[:,-1]

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, stratify = y, random_state=42)
X_train, X_valid, y_train, y_valid = train_test_split(X_train, y_train, test_size = 0.1, random_state=42)

In [77]:
numeric_cols = ['이동거리', '출발예측시간', '도착예측시간']

scaler = StandardScaler()
X_train[numeric_cols] = scaler.fit_transform(X_train[numeric_cols])
X_valid[numeric_cols] = scaler.transform(X_valid[numeric_cols])
X_test[numeric_cols] = scaler.transform(X_test[numeric_cols])

print(X_train.shape, y_train.shape)
print(X_valid.shape, y_valid.shape)
print(X_test.shape, y_test.shape)

(157136, 14) (157136,)
(17460, 14) (17460,)
(43650, 14) (43650,)


In [83]:
from imblearn.under_sampling import RandomUnderSampler

# RandomUnderSampler 객체 생성
rus = RandomUnderSampler(random_state=42)
X_train_under, y_train_under = rus.fit_resample(X_train, y_train)

# 결과 출력
print('언더샘플링 적용 전 학습용 피처/레이블 데이터 세트 : ', X_train.shape, y_train.shape)
print('언더샘플링 적용 후 학습용 피처/레이블 데이터 세트 : ', X_train_under.shape, y_train_under.shape)
print('언더샘플링 적용 후 레이블 분포 : \n', pd.Series(y_train_under).value_counts())

언더샘플링 적용 전 학습용 피처/레이블 데이터 세트 :  (157136, 14) (157136,)
언더샘플링 적용 후 학습용 피처/레이블 데이터 세트 :  (55406, 14) (55406,)
언더샘플링 적용 후 레이블 분포 : 
 항공편지연여부
0.0    27703
1.0    27703
Name: count, dtype: int64


In [53]:
data.info()

<class 'pandas.core.frame.DataFrame'>
Index: 218246 entries, 4 to 854941
Data columns (total 15 columns):
 #   Column    Non-Null Count   Dtype  
---  ------    --------------   -----  
 0   월         218246 non-null  int64  
 1   일         218246 non-null  int64  
 2   출발예측시간    218246 non-null  float64
 3   도착예측시간    218246 non-null  float64
 4   결항여부      218246 non-null  int64  
 5   경유여부      218246 non-null  int64  
 6   출발공항고유코드  218246 non-null  int64  
 7   출발공항고유ID  218246 non-null  int64  
 8   출발공항_주    218246 non-null  int64  
 9   도착공항고유코드  218246 non-null  int64  
 10  도착공항공유ID  218246 non-null  int64  
 11  도착공항_주    218246 non-null  int64  
 12  이동거리      218246 non-null  int64  
 13  항공사       218246 non-null  int64  
 14  항공편지연여부   218246 non-null  float64
dtypes: float64(3), int64(12)
memory usage: 26.6 MB


In [23]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout
from tensorflow.keras.callbacks import EarlyStopping

In [102]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

# 모델 구축
model = Sequential([
    Dense(32, input_dim=X_train.shape[1], activation='relu'),
    Dropout(0.1),
    Dense(16, activation='relu'),
    Dropout(0.1),
    Dense(4, activation='relu'),
    Dense(1, activation='sigmoid')
])

# 모델 컴파일
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

# 모델 훈련
model.fit(X_train, y_train, epochs=3, batch_size=10, validation_data=(X_valid, y_valid))

# 모델 평가
y_pred = (model.predict(X_test) > 0.5).astype("int32")

# 정확도, 정밀도, 재현율, F1 스코어 계산
accuracy = accuracy_score(y_test, y_pred)
precision = precision_score(y_test, y_pred)
recall = recall_score(y_test, y_pred)
f1 = f1_score(y_test, y_pred)

# 결과 출력
print(f'Test Accuracy: {accuracy:.2f}')
print(f'Test Precision: {precision:.2f}')
print(f'Test Recall: {recall:.2f}')
print(f'Test F1 Score: {f1:.2f}')


Epoch 1/3
15714/15714 [==============================] - 30s 2ms/step - loss: 2.6031 - accuracy: 0.8222 - val_loss: 0.4692 - val_accuracy: 0.8214
Epoch 2/3
15714/15714 [==============================] - 28s 2ms/step - loss: 0.4665 - accuracy: 0.8237 - val_loss: 0.4692 - val_accuracy: 0.8214
Epoch 3/3
1365/1365 [==============================] - 1s 975us/step
Test Accuracy: 0.82
Test Precision: 0.00
Test Recall: 0.00
Test F1 Score: 0.00


/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [84]:
# 모델 구축
model = Sequential([
    Dense(64, input_dim=X_train.shape[1], activation='relu'),
    Dropout(0.1),
    Dense(32, activation='relu'),
    Dropout(0.1),
    Dense(16, activation='relu'),
    Dense(1, activation='sigmoid')
])

# 모델 컴파일
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# 모델 훈련
model.fit(X_train, y_train, epochs=5, batch_size=10, validation_data=(X_valid, y_valid))

# 모델 평가
y_pred = (model.predict(X_test) > 0.5).astype("int32")

# 정확도, 정밀도, 재현율, F1 스코어 계산
accuracy = accuracy_score(y_test, y_pred)
precision = precision_score(y_test, y_pred)
recall = recall_score(y_test, y_pred)
f1 = f1_score(y_test, y_pred)

# 결과 출력
print(f'Validation Accuracy: {accuracy:.2f}')
print(f'Validation Precision: {precision:.2f}')
print(f'Validation Recall: {recall:.2f}')
print(f'Validation F1 Score: {f1:.2f}')

Epoch 1/5
15714/15714 [==============================] - 30s 2ms/step - loss: 1.9061 - accuracy: 0.7989 - val_loss: 0.4693 - val_accuracy: 0.8214
Epoch 2/5
15714/15714 [==============================] - 29s 2ms/step - loss: 0.5624 - accuracy: 0.8237 - val_loss: 0.4693 - val_accuracy: 0.8214
Epoch 3/5
15714/15714 [==============================] - 29s 2ms/step - loss: 0.4733 - accuracy: 0.8237 - val_loss: 0.4694 - val_accuracy: 0.8214
Epoch 4/5
15714/15714 [==============================] - 34s 2ms/step - loss: 0.4774 - accuracy: 0.8237 - val_loss: 0.4692 - val_accuracy: 0.8214
Epoch 5/5
1365/1365 [==============================] - 1s 1ms/step
Validation Accuracy: 0.82
Validation Precision: 0.00
Validation Recall: 0.00
Validation F1 Score: 0.00


/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [93]:
accuracy = accuracy_score(y_test, y_pred)
precision = precision_score(y_test, y_pred, zero_division=1)  # zero_division 설정 추가
recall = recall_score(y_test, y_pred, zero_division=1)  # zero_division 설정 추가
f1 = f1_score(y_test, y_pred, zero_division=1)  # zero_division 설정 추가

# 결과 출력
print(f'Validation Accuracy: {accuracy:.2f}')
print(f'Validation Precision: {precision:.2f}')
print(f'Validation Recall: {recall:.2f}')
print(f'Validation F1 Score: {f1:.2f}')

Validation Accuracy: 0.82
Validation Precision: 1.00
Validation Recall: 0.00
Validation F1 Score: 0.00


In [94]:
from sklearn.metrics import confusion_matrix

cm = confusion_matrix(y_test, y_pred)

In [95]:
cm

array([[35944,     0],
       [ 7706,     0]])

In [ ]:
!pip install tensorflow

In [27]:
!pip install tensorflow==2.12.0

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 585.9/585.9 MB 1.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 68.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.1/17.1 MB 55.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.6/5.6 MB 74.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 440.7/440.7 kB 48.7 MB/s eta 0:00:00
  Attempting uninstall: tensorflow-estimator
    Found existing installation: tensorflow-estimator 2.15.0
    Uninstalling tensorflow-estimator-2.15.0:
      Successfully uninstalled tensorflow-estimator-2.15.0
  Attempting uninstall: numpy
    Found existing installation: numpy 1.25.2
    Uninstalling numpy-1.25.2:
      Successfully uninstalled numpy-1.25.2
  Attempting uninstall: keras
    Found existing installation: keras 2.15.0
    Uninstalling keras-2.15.0:
      Successfully uninstalled keras-2.15.0
  Attempting uninstall: gast
    Found existing installation: gast 0.5.4
    Uninst

In [24]:
from sklearn.model_selection import GridSearchCV
from tensorflow.keras.wrappers.scikit_learn import KerasClassifier

# 모델 생성 함수
def create_model(dropout_rate=0.0, neurons=64):
    model = Sequential([
        Dense(neurons, input_dim=X_train.shape[1], activation='relu'),
        Dropout(dropout_rate),
        Dense(neurons // 2, activation='relu'),
        Dropout(dropout_rate),
        Dense(neurons // 4, activation='relu'),
        Dense(1, activation='sigmoid')
    ])
    model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
    return model

# 모델 생성기 생성
model = KerasClassifier(build_fn=create_model, epochs=50, batch_size=32, verbose=0)

# 하이퍼파라미터 그리드 정의
param_grid = {
    'dropout_rate': [0.1, 0.2, 0.3, 0.4, 0.5],
    'neurons': [32, 64, 128]
}

# 그리드 서치 객체 생성
grid = GridSearchCV(estimator=model, param_grid=param_grid, scoring=['accuracy', 'recall', 'precision', 'f1'], refit='accuracy', cv=3)

# 그리드 서치 수행
grid_result = grid.fit(X_train, y_train)

# 결과 출력
print("최적 파라미터: ", grid_result.best_params_)
print("최적 점수 (정확도): ", grid_result.best_score_)

# 최적 모델 추출
best_model = grid_result.best_estimator_

# 테스트 데이터로 모델 평가
y_pred = best_model.predict(X_test)

# 정확도, 정밀도, 재현율, F1 스코어 계산
accuracy = accuracy_score(y_test, y_pred)
precision = precision_score(y_test, y_pred, zero_division=1)
recall = recall_score(y_test, y_pred, zero_division=1)
f1 = f1_score(y_test, y_pred, zero_division=1)

# 결과 출력
print(f'테스트 정확도: {accuracy:.2f}')
print(f'테스트 정밀도: {precision:.2f}')
print(f'테스트 재현율: {recall:.2f}')
print(f'테스트 F1 스코어: {f1:.2f}')

<ipython-input-24-c9b1410cc48e>:18: DeprecationWarning: KerasClassifier is deprecated, use Sci-Keras (https://github.com/adriangb/scikeras) instead. See https://www.adriangb.com/scikeras/stable/migration.html for help migrating.
  model = KerasClassifier(build_fn=create_model, epochs=50, batch_size=32, verbose=0)


KeyboardInterrupt: 